In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Movielens model
In this notebook we build a movie recomandation model using movielens dataset.
The model uses entity embeddings for categorical variables from [this paper](https://arxiv.org/abs/1604.06737) to embedd users and movies into two 50 dimensional spaces.
Hence we have 3 methods to recomand movies: 
1. evaluating the model,
2. look at close neighbords of a movie in the embedding space,
3. look at close neighbords of a user in the embedding space, and recomand those top movies.

The code woarks on Linux and Windows.

## Imports

In [2]:
import re
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
from pathlib import Path
import seaborn as sns

In [3]:
from movienet import MovieNet

Using TensorFlow backend.


In [4]:
from sklearn.model_selection import train_test_split

## Loading datasets

In [5]:
PATH = Path("data/ml-latest-small")
PATH = Path("data/ml-20m")

In [6]:
ratings_raw = pd.read_csv(PATH/"ratings.csv")
movies_raw = pd.read_csv(PATH/"movies.csv")

In [7]:
display(ratings_raw.head())
display(movies_raw.head())

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Features engineering
The datasets are clean, we only creating dictionnaries to convert ids and indexes.

In [8]:
ratings_train = ratings_raw.copy()

Creating dictionaries to convert userId and movieId into index and vice versa.

In [9]:
users_uniq = ratings_train.userId.unique()
user2idx = {o:i for i,o in enumerate(users_uniq)}
idx2user = {i:o for i,o in enumerate(users_uniq)}
ratings_train.userId = ratings_train.userId.apply(lambda x: user2idx[x])

movies_uniq = ratings_train.movieId.unique()
movie2idx = {o:i for i,o in enumerate(movies_uniq)}
idx2movie = {i:o for i,o in enumerate(movies_uniq)}
ratings_train.movieId = ratings_train.movieId.apply(lambda x: movie2idx[x])

n_users = int(ratings_train.userId.nunique())
n_movies = int(ratings_train.movieId.nunique())
n_users, n_movies

(138493, 26744)

In [10]:
ratings_train.head()

,userId,movieId,rating,timestamp
0,0,0,3.5,1112486027
1,0,1,3.5,1112484676
2,0,2,3.5,1112484819
3,0,3,3.5,1112484727
4,0,4,3.5,1112484580


In [11]:
def save_obj(obj, name):  
    with open(Path("model")/str(name+'.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

save_obj(user2idx, "user2idx")
save_obj(idx2user, "idx2user")
save_obj(movie2idx, "movie2idx")
save_obj(idx2movie, "idx2movie")

## Keras Model
The model works as follows:
1. Embedds the user and movie id.
2. Concanate the user embedding, movie embedding and the weighted rating into one vector.
3. Passes to linear layers with dropout.

The architecture takes as parameters the embedding size, the size of hidden layers, and the dropout probability associate to them.

Spliting data into train and validation sets.

In [43]:
movie_model = MovieNet(n_users, n_movies)
movie_model.build_model(emb_size=[50, 50], hl=[70, 10], drop=[0.4, 0.3])

In [44]:
X = ratings_train.drop(['timestamp', 'rating'], axis=1)
y = ratings_train['rating']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
len(X_train), len(X_valid), len(y_train), len(y_valid)

(16000210, 4000053, 16000210, 4000053)

It's important that every movie are in the training set to have trained embedding of each of them.

In [45]:
len(X_train["movieId"].unique()), n_movies, n_movies - len(X_train["movieId"].unique())

(25813, 26744, 931)

In [46]:
miss_movies = ratings_train[~ratings_train.movieId.isin(X_train["movieId"].unique())]["movieId"].unique()

In [47]:
concat = pd.DataFrame()
for i in miss_movies:
    concat = concat.append(ratings_train[ratings_train.movieId == i].sample(1))

In [48]:
concat.head()

,userId,movieId,rating,timestamp
13786175,95253,2740,2.5,1292515144
72739,511,7451,4.0,1337086759
146083,978,9658,0.5,1420447834
168531,1140,9916,3.0,1424079101
220497,1517,10294,1.5,1391699625


In [49]:
X_valid.drop(concat.index, axis=0, inplace=True)
y_valid.drop(concat.index, axis=0, inplace=True)

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
X_train = pd.concat([X_train, concat.drop(["rating", "timestamp"], axis=1)])
y_train = pd.concat([y_train, concat["rating"]])

In [51]:
len(X_train["movieId"].unique()), n_movies

(26744, 26744)

In [53]:
movie_model.fit(X_train, y_train, X_valid, y_valid, epochs=5, batch_size=512)

Train on 16001141 samples, validate on 3999122 samples
Epoch 1/5
16001141/16001141 [==============================] - 500s 31us/step - loss: 0.7826 - rmse: 0.8837 - val_loss: 0.7054 - val_rmse: 0.8393
Epoch 2/5
16001141/16001141 [==============================] - 497s 31us/step - loss: 0.7311 - rmse: 0.8544 - val_loss: 0.6880 - val_rmse: 0.8288
Epoch 3/5
16001141/16001141 [==============================] - 500s 31us/step - loss: 0.7136 - rmse: 0.8441 - val_loss: 0.6792 - val_rmse: 0.8235
Epoch 4/5
16001141/16001141 [==============================] - 498s 31us/step - loss: 0.7002 - rmse: 0.8362 - val_loss: 0.6730 - val_rmse: 0.8198
Epoch 5/5
16001141/16001141 [==============================] - 497s 31us/step - loss: 0.6902 - rmse: 0.8302 - val_loss: 0.6658 - val_rmse: 0.8154


In [54]:
movie_model.save_model(name="movie_model")

In [55]:
movie_model.fit(X_train, y_train, X_valid, y_valid, epochs=1, batch_size=512)

Train on 16001141 samples, validate on 3999122 samples
Epoch 1/1
16001141/16001141 [==============================] - 498s 31us/step - loss: 0.6816 - rmse: 0.8250 - val_loss: 0.6622 - val_rmse: 0.8131


In [56]:
movie_model.save_model(name="movie_model")

In [58]:
movie_model.fit(X_train, y_train, X_valid, y_valid, epochs=12, batch_size=128)

Train on 16001141 samples, validate on 3999122 samples
Epoch 1/12
16001141/16001141 [==============================] - 1958s 122us/step - loss: 0.6871 - rmse: 0.8265 - val_loss: 0.6639 - val_rmse: 0.8123
Epoch 2/12
16001141/16001141 [==============================] - 1954s 122us/step - loss: 0.6815 - rmse: 0.8231 - val_loss: 0.6648 - val_rmse: 0.8130
Epoch 3/12
16001141/16001141 [==============================] - 1959s 122us/step - loss: 0.6755 - rmse: 0.8194 - val_loss: 0.6632 - val_rmse: 0.8121
Epoch 4/12
16001141/16001141 [==============================] - 1959s 122us/step - loss: 0.6703 - rmse: 0.8162 - val_loss: 0.6538 - val_rmse: 0.8061
Epoch 5/12
16001141/16001141 [==============================] - 1953s 122us/step - loss: 0.6660 - rmse: 0.8137 - val_loss: 0.6498 - val_rmse: 0.8036
Epoch 6/12
16001141/16001141 [==============================] - 1953s 122us/step - loss: 0.6626 - rmse: 0.8115 - val_loss: 0.6507 - val_rmse: 0.8041
Epoch 7/12
16001141/16001141 [=====================

In [59]:
movie_model.save_model(name="movie_model")

The current state of the art models uses either [matrix factorization](https://docs.treasuredata.com/articles/hivemall-movielens20m-fm) with RMSE of 0.80 or [autoencoders](https://arxiv.org/pdf/1606.07659.pdf) with RMSE of 0.81.

**Our model has a RMSE of ~0.80, on par with state of the art models.
The approach of entity embeddings is simple but efficient.**